<a href="https://colab.research.google.com/github/satfail/AI-Reading-Materials/blob/master/Xception_FullDataSet_CargaCheckPoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargamos las imágenes el Drive e importamos Tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/MyDrive/Celulas"

#raiz
PATH = "/content/drive/My Drive"

#ipunt

INPATH = PATH + '/Celulas'



#checkpoints

CPATH = PATH + '/checkpointsCelulas'

In [ ]:
# Reestablecer tf a la versión 2.4.1

!pip uninstall tensorflow -y
!pip install  tensorflow==2.4.1 


# Importamos las librerías generales para el desarrollo

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import AveragePooling2D

print(tf.__version__)

# Cargamos el set de Datos y Preprocesado
*   Defininmos el ImageDataGenerator para aumentar el dataset, con zoom, flip,rotación..
*  Hacemos plot para compronbar como quedan las imágenes generadas

In [ ]:
import pathlib
data_train = pathlib.Path(INPATH + '/entrenamiento/') 
count = len(list(data_train.glob('*/*.tiff')))
print('Entrenamiento : ' + str(count))
data_test = pathlib.Path(INPATH + '/test/') 
count = len(list(data_test.glob('*/*.tiff')))
print('Test : ' + str(count))
data_train

In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator


batch_size = 32
img_height = 224
img_width = 224

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.1) # set validation split

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_train,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical',
    subset='training',
    seed=42) # set as training data

validation_generator = train_datagen.flow_from_directory(
    data_train, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical',
    subset='validation',
    seed=42) # set as validation data

test_generator = test_datagen.flow_from_directory(INPATH + '/test',
                                                target_size=(224, 224),
                                                batch_size=1,
                                                shuffle=False,
                                             class_mode='categorical')



In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()
    
    
augmented_images = [train_generator[1][0][0] for i in range(5)]
plotImages(augmented_images)

# Cargamos el modelo Xception

* Usamos los pesos de imagenet
* No incluimos la capa final, ya que debemos ajustarla a nuestro problema
* Damos la forma de entrada de nuestros datos

In [ ]:
from tensorflow.keras.layers import Dropout, BatchNormalization
#from keras.applications.xception import Xception
from tensorflow.python.keras.applications.xception import Xception

baseModel = Xception(weights="imagenet", include_top=False,
	input_shape =  (224, 224, 3) )

# Fine Tuning

* Ajustamos la cabeza de la red para que nos de una salida categórica de 4 elementos acorde con nuestro problema

In [ ]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(4, activation="softmax")(headModel)

In [ ]:
from tensorflow.python.keras.models import Model
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
model.summary()

In [ ]:
print("Number of layers in the base model: ", len(model.layers))

# Fine Tuning

Congelamos las primeras 40 capas del modelo, para utilizar el mapa de caracteristicas general que ha cargado de los pesos.


In [ ]:
##Solo
for layer in model.layers[:30]:
    layer.trainable = False

for layer in model.layers[:]:
  print(layer.trainable)

# Calculamos los pasos por epoca y validación

In [ ]:
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

print(steps_per_epoch)
print(validation_steps)

# Cargamos el modelo del checkpoint

In [ ]:
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
optimizer = tf.keras.optimizers.Adam (lr=0.001)

checkpoint_filepath = CPATH + '/fulldata/'

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.load_weights(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    save_freq = 30,
    save_best_only=False)

reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

# Opcional : Seguir con el entrenamiento

In [ ]:
history = model.fit(train_generator,
          steps_per_epoch=steps_per_epoch,
          epochs=1500,
          validation_data=validation_generator,
          validation_steps=validation_steps,
          callbacks=[reduce_lr,model_checkpoint_callback,earlystopping],verbose=1)

# Evaluamos el entrenamiento

In [ ]:
model.evaluate(x=validation_generator,
steps=validation_steps)

# Predicción de resultados

In [ ]:
test_predict = model.predict(test_generator, steps = test_generator.n // 1, verbose =1)

In [ ]:
test_predict.shape

In [ ]:
test_predict

In [ ]:
predict = []

for i in test_predict:
  predict.append(int(np.argmax(i)))

predict = np.asarray(predict)

In [ ]:
np.asarray(predict)


# Métricas de exactitud y Matriz de Confusión

In [ ]:
# Obtenemos la tasa de acierto del modelo
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_generator.classes, np.asarray(predict))
accuracy

0.75

In [ ]:
test_generator.class_indices

In [ ]:
# Representamos la matriz de confusión
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(test_generator.classes, predict)
plt.figure(figsize = (7,7))
sns.heatmap(cm, annot=True)